Desenvolvido por: Heitor santos
Empresa: Dataex
Códificação para tratar as macros do Tesouraria fase 2. Realizando a leitura de arquivos do Google Cloud Storage, tratar as informações e realizar o insert no Google Big Query para seguir com o processo de cargar.

#Definição dos importes do código.

In [266]:
import math
import pandas as pd
#import scipy as interpolate
import numpy as np
import scipy.stats

from datetime import datetime
from scipy.stats import norm
from scipy import interpolate
from numpy.random import normal
from google.cloud import bigquery,storage
from google.oauth2 import service_account


Variaveis pré definidas

In [267]:

# Caminho do arquivo chave para o GCP
key_path = "D:\MeusDocumentos\Clientes\Dataex\BancoABC\dev-terceiros.json" 

# Query de curvas VOL_IND
v_QUERY_STG_TB_VOL_IND = """
    SELECT Data_ref, Data_proc, Data, Delta, PER_1, PER_10, PER_25, PER_37, PER_50, PER_63, PER_75, PER_90, PER_99 FROM `abc-pipeline-dev-terceiros.DS_RISCOS_RESULTADOS.STG_TB_VOL_IND` ORDER BY Delta 
    """

# Definição do nome do Bucket a ser utilizado para conectar no GCP    
v_bucket_name = "abc-pipeline-dev-terceiros_cloudbuild" 

# Definição do nome do arquivo com o caminho e sem o bucket a ser buscado para conectar no GCP
v_source_file_ic = "Arquivos/TEMP"  

# Definição do nome do arquivo que será realizado o download.
v_source_file_name = "/IBOVOPT_VANILLA_TEMP.CSV"

# Definição do caminho nome do arquivo que será realizado o download.
v_target_file_ic = "D:\MeusDocumentos\Clientes\Dataex\BancoABC\Tesouraria Fase 02\DEV - Macros\python\TesourariaFase2"


# Definição do nome do arquivo que será gravado no destino.
v_target_file_name = "\IBOVOPT_VANILLA_TEMP.CSV"

# Definição do nome do arquivo que será gravado realizando o upload no Cloud Storage.
v_upload_file_name = "/IBOVOPT_VANILLA_UPLOAD.CSV"

Realiza download do arquivo no Cloud Storage

In [268]:
def download_file_cs(bucket_name, source_blob_name, destination_file_name):
    
    try:
        
        # Abre uma variavel com o caminho da chave do GCP
        storage_client = storage.Client.from_service_account_json(key_path)

        # Define o nome do bucket a ser conectado.
        bucket = storage_client.bucket(bucket_name)    

        # Caminho do arquivo com o seu caminho
        blob = bucket.blob(source_blob_name)
        
        # Realiza o download do arquivo e 
        blob.download_to_filename(str(destination_file_name))
        
    except Exception as e:
        print(e)

In [269]:
def upload_file_cs(bucket_name, target_destination_name, source_file_name):

    try:
        
        # Abre uma variavel com o caminho da chave do GCP
        storage_client = storage.Client.from_service_account_json(key_path)
        
        bucket  = storage_client.bucket(bucket_name)
        
        blob = bucket.blob(target_destination_name)
        
        blob.upload_from_filename(source_file_name)
        
    except Exception as e:
        print(e)

In [270]:
def create_dataframe_to_csv(dataframe, target_file_ic, file_name):
    
    # Criar um CSV em um diretorio e com um nome especifico para ser enviado para o Cloud Storage
    dataframe.to_csv(target_file_ic + file_name, index=False)

Realiza a leitura do arquivo gerado do CS, realizando o formato dos campos de datas

In [271]:
def read_file_csv(read_file):

    try:
        
        # Cria um dataframe para ler e registrar os dados do arquivo. delimitado por ";".
        df = pd.read_csv(read_file, encoding='ANSI',  delimiter=',')

        # Formata cada um dos campos de datas abaixo.
        df['Data_base_d0']= pd.to_datetime(df['Data_base_d0'].str.slice(0, 10), format='%m/%d/%Y')
        df['Data_base_d1']= pd.to_datetime(df['Data_base_d1'].str.slice(0, 10), format='%m/%d/%Y')
        df['Data_base_m1']= pd.to_datetime(df['Data_base_m1'].str.slice(0, 10), format='%m/%d/%Y')
        df['Dia_util_d0']= pd.to_datetime(df['Dia_util_d0'].str.slice(0, 10), format='%m/%d/%Y')
        df['Dia_util_d1']= pd.to_datetime(df['Dia_util_d1'].str.slice(0, 10), format='%m/%d/%Y')
        df['Dia_util_m1']= pd.to_datetime(df['Dia_util_m1'].str.slice(0, 10), format='%m/%d/%Y')
        df['Trade_Date']= pd.to_datetime(df['Trade_Date'].str.slice(0, 10), format='%m/%d/%Y')
        df['Initial_Date']= pd.to_datetime(df['Initial_Date'].str.slice(0, 10), format='%m/%d/%Y')
        df['Fixing_Date']= pd.to_datetime(df['Fixing_Date'].str.slice(0, 10), format='%m/%d/%Y')
        df['Vencimento']= pd.to_datetime(df['Vencimento'].str.slice(0, 10), format='%m/%d/%Y')
        df['Data_Liquidacao']= pd.to_datetime(df['Data_Liquidacao'].str.slice(0, 10), format='%m/%d/%Y')
        
        
        if df['d1_data'].isna:
            None
        else:
            df['d1_data']= pd.to_datetime(df['d1_data'].str.slice(0, 10), format='%m/%d/%Y')

        if df['d1_data_vertice_1'].isna:
            None
        else:
            df['d1_data_vertice_1']= pd.to_datetime(df['d1_data_vertice_1'].str.slice(0, 10), format='%m/%d/%Y')            

        if df['d1_data_vertice_2'].isna:
            None
        else:
            df['d1_data_vertice_2']= pd.to_datetime(df['d1_data_vertice_2'].str.slice(0, 10), format='%m/%d/%Y')        
        

        return df

    except Exception as e:
        
        print(e)

In [272]:
df_File = read_file_csv(v_target_file_ic + v_target_file_name)

Abre conexão com o BQ e cria o dataframe com a consulta com consulta de SELECT

In [273]:
def open_cnx_bq(sql):
    try:
        
        # Habilita a credencial no GCP com o Chave e o caminho da plataforma.
        credentials = service_account.Credentials.from_service_account_file(
            key_path,
            scopes=["https://www.googleapis.com/auth/cloud-platform"],
        )

        # Client de acesso ao BQ, com as credenciais e o projeto. Abre a conexão com o BQ.
        client = bigquery.Client(
            credentials=credentials,
            project=credentials.project_id
        )
        
        # Execução da query tendo como o retorno um dataframe com os dados.
        rows = (
            client.query(sql)
            .result()
            .to_dataframe()
        )

        df = pd.DataFrame(rows)
        
        # Fecha a conexão com o BQ
        client.close()
        
        # Dataframe a ser retornado.
        return df
    
    except Exception as e:
        print(e)


Função de treinamento da interpolação Cúbica

In [274]:
def BASE_TREINO(df):

    # Realiza uma leitura do dataframe apenas da quarta coluna em diante.
    v_TABELA_VOL = df[df.columns[4:]].to_numpy()

    # Realiza uma leitura do dataframe apenas da terceira coluna em diante.
    x = df[df.columns[3]].to_numpy()
    
    y = [0.01, 0.10, 0.25, 0.37, 0.50, 0.63, 0.75, 0.90, 0.99]

    # Gera uma variavel com o treinamento da interpolação cúbica.
    f = interpolate.interp2d(y, x, v_TABELA_VOL, kind='cubic')

    return f



As Funções CAL_D1 e CALC_D2. Criado baseado em 3 funções VBA do Excel (DERIVATIVOS_OPCAO_IBOV, DERIVATIVOS_VOL_OPCAO_IBOV, DERIVATIVOS_DELTA_OPCAO_IBOV)

In [275]:
def CALC_D1(FWD1, strike1, Vol1, T21):
    
    D1 = (math.log(FWD1 / strike1) + ((math.pow(Vol1, 2) / 2) * T21)) / (Vol1 * math.sqrt(T21)) # Calculo baseado nas funções VBA do Excel
    
    return D1

In [276]:
def CALC_D2(FWD2, strike2, Vol2, T22):
    
    D2 = math.log(FWD2 / strike2) - ((math.pow(Vol2, 2) / 2) * T22) / (Vol2 * math.sqrt(T22)) # Calculo baseado nas funções VBA do Excel
    
    return  D2

Replicação do código VBA do Excel - DERIVATIVOS_OPCAO_IBOV

In [277]:
def DERIVATIVOS_OPCAO_IBOV(TIPO, FWD, strike, Vol, PRE, T):
    
    if T > 1:
        
        if TIPO == "0":

            DOI = 0
            
        else:
            
            T2  = T/252
            
            r = math.log(1+PRE)
            
            d1 = CALC_D1(FWD, strike, Vol, T)
    
            d2 = CALC_D2(FWD, strike, Vol, T)
            
            if TIPO == "CALL":
                DOI = math.exp(-r * T2) * (FWD * normal(d1) - strike * normal(d2))
            elif TIPO == "PUT":
                DOI = math.exp(-r * T2) * (FWD * normal(-d1) - strike * normal(-d2))
            else:
                DOI = "ERRO"
        
    else:
        
        DOI = abs(FWD - strike)

        if (TIPO == "CALL" and (FWD - strike) < 0): DOI = 0

        if (TIPO == "PUT" and (strike - FWD) < 0): DOI = 0

        if (TIPO == "0"): DOI = 0
        
    return DOI  

Replicação do código VBA do Excel - DERIVATIVOS_VOL_OPCAO_IBOV

In [278]:
def DERIVATIVOS_VOL_OPCAO_IBOV(FWD, strike, Vol, PRE, T, treino):

    if strike != 0 and T > 1:

        T2 = T / 252
        
        #print("T2:", T2)
        
        r = math.log(1 + PRE)
        
        #print("r:", r)  
          
        VOL2 = Vol + 0.00002
        
        #print("PRIMEIRO VOL2:", VOL2)
        
        i = 0
        while abs(VOL2 - Vol) > 0.00001:

            Vol = VOL2
            
            d1 = CALC_D1(FWD, strike, Vol, T2)
            
            delta = math.exp(-r * T2) * (1-scipy.stats.norm(d1, 1).cdf(0))
            
            # Com o Treinamento da Interpolação Cúbica realizada. E passado os dados do excel para geração final dos campos do excel
            VOL2 = treino(delta, [T])

            i = i + 1
            if i == 1000:
                exit()

        return VOL2

    return 0


Replicação do código VBA do Excel - DERIVATIVOS_DELTA_OPCAO_IBOV

In [279]:
def DERIVATIVOS_DELTA_OPCAO_IBOV(TIPO, FWD, strike, Vol, PRE, T):
    
    if T > 1:
        if TIPO == 0:
            DDOI = 0
        
        else:
            
            T2 = T / 252
            r = math.log(1 + PRE)
            
            d1 = CALC_D1(FWD, strike, Vol, T)
            d2 = CALC_D2(FWD, strike, Vol, T)
        
            if TIPO == "CALL":
                DDOI = math.exp(-r * T2) * normal(d1)
            elif TIPO == "PUT":
                DDOI = math.exp(-r * T2) * normal(d1) - 1
            Else: DDOI = "ERRO"

    else:
        DDOI = 0

Realiza a chamada para o download do arquivo do CS

In [280]:
download_file_cs(v_bucket_name, v_source_file_ic + v_source_file_name, v_target_file_ic + v_target_file_name)

#Leitura do arquivo baixado do Cloud Storage que foi gerado pelo Informatica Cloud Data Integration(CDI): Pasta: abc_tesouraria_riscos_e_resultados\Fase2 MAPA: map_stg_tb_ibovopt_vanilla_1

In [281]:
df_File = read_file_csv(v_target_file_ic + v_target_file_name)

Abre a conexão com o BQ para realizar o select e gerar o Dataframe da tabela STG_TB_VOL_IND

In [282]:
df_DVOI = open_cnx_bq(v_QUERY_STG_TB_VOL_IND)

Realiza a chamada do treino dos dados

In [283]:
v_BASE_TREINO_DVOI = BASE_TREINO(df_DVOI)

In [284]:
print(df_DVOI)

      Data_ref           Data_proc        Data  \
0   2021-10-27 2022-03-22 17:23:44  2021-11-17   
1   2021-10-27 2022-03-22 17:23:44  2021-12-15   
2   2021-10-27 2022-03-22 17:23:44  2022-01-12   
3   2021-10-27 2022-03-22 17:23:44  2022-02-16   
4   2021-10-27 2022-03-22 17:23:44  2022-03-16   
5   2021-10-27 2022-03-22 17:23:44  2022-04-13   
6   2021-10-27 2022-03-22 17:23:44  2022-06-15   
7   2021-10-27 2022-03-22 17:23:44  2022-07-13   
8   2021-10-27 2022-03-22 17:23:44  2022-08-17   
9   2021-10-27 2022-03-22 17:23:44  2022-09-14   
10  2021-10-27 2022-03-22 17:23:44  2022-10-13   
11  2021-10-27 2022-03-22 17:23:44  2022-12-14   
12  2021-10-27 2022-03-22 17:23:44  2023-01-18   
13  2021-10-27 2022-03-22 17:23:44  2023-02-01   
14  2021-10-27 2022-03-22 17:23:44  2024-04-15   

                                         Delta  \
0    13.00000000000000000000000000000000000000   
1    33.00000000000000000000000000000000000000   
2    53.00000000000000000000000000000000000000   

Calculo da média da coluna Delta_50, da tabela abc-pipeline-dev-terceiros.DS_RISCOS_RESULTADOS.STG_TB_VOL_IND 

In [285]:
v_media_delta_50 = np.mean(df_DVOI["PER_50"])

Loop para que possa ser gerada as informações das colunas a serem calculadas.

In [286]:

for i in range(len(df_File)):

    #VARIAVEIS USADAS EM CONDIÇÕES: (IF)
    v_Data_base_d0 = df_File["Dia_util_d0"][i]
    v_Data_base_d1 = df_File["Dia_util_d1"][i]
    v_Data_base_m1 = df_File["Dia_util_m1"][i]
    v_Fixing_Date = df_File["Fixing_Date"][i]
    v_Trade_Date = df_File["Trade_Date"][i]
    v_Liquidacao_Antecipada_x = df_File["Liquidacao_Antecipada_x"][i]
    v_Data_Liquidacao = df_File["Data_Liquidacao"][i]
    
    #VARIAVEIS USADAS COMO PARAMETROS DAS FUNÇÕES: DERIVATIVOS_OPCAO_IBOV e DERIVATIVOS_VOL_OPCAO_IBOV
    v_Tipo = df_File["Call_Put"][i]
    v_Premio_Tesouraria_Liq_curva = df_File["Premio_Tesouraria_Liq_curva"][i]
    v_m_indice_futuro = df_File["Strike"][i]
    v_m_data_base_vencimento_du = df_File["m_taxa_pre_brl_vencimento_perc"][i]
    v_T = df_File["m_data_base_vencimento_du"][i]
    v_FWD = df_File["m_indice_futuro"][i]
    v_strike = df_File["Strike"][i]
    v_Vol = v_media_delta_50
    v_PRE = df_File["m_taxa_pre_brl_vencimento_perc"][i]
    V_Notional = df_File["Notional"][i]

    #Inicio do calculo do campo:  m_vol_perc    
    if (v_Fixing_Date  is None or v_Data_Liquidacao is None ):
        v_m_vol_perc = 0
    else:
        if (v_Tipo is None or v_Data_base_d0 >= v_Fixing_Date and (v_Liquidacao_Antecipada_x == "x" and v_Data_Liquidacao < v_Data_base_d0)):
            v_m_vol_perc = 0
        else:
            v_m_vol_perc = DERIVATIVOS_VOL_OPCAO_IBOV(v_FWD, v_strike, v_Vol, v_PRE, v_T, v_BASE_TREINO_DVOI) 
    
    v_result_m_vol_perc = v_m_vol_perc
    #Fim do calculo do campo:  m_vol_perc 
     
    #Inicio do calculo do campo:  m_preco_opcao
    if ( v_Trade_Date is None ):
        v_m_preco_opcao = 0
    else:
        if ( v_Data_base_d0 < v_Trade_Date):
            v_m_preco_opcao = 0
        else:
            if (v_Liquidacao_Antecipada_x == "x" and v_Data_Liquidacao <= v_Data_base_d0):
                v_m_preco_opcao = v_Premio_Tesouraria_Liq_curva
            else:
                v_m_preco_opcao = DERIVATIVOS_OPCAO_IBOV(v_Tipo, v_m_indice_futuro, v_strike, v_m_vol_perc, 0, v_m_data_base_vencimento_du)
    #Fim do calculo do campo:  m_preco_opcao 
    
    # Registra no Dataframe o retorno do calculo realizado pelas funções.
    df_File.loc[[i], "m_vol_perc"] = v_m_vol_perc
    df_File.loc[[i], "m_preco_opcao"] = v_m_preco_opcao
    df_File.loc[[i], "pl_ltd_opcao"] = df_File["Notional"][i] / df_File["Strike"][i] * (v_m_preco_opcao - df_File["m_premio_corrigido"][i])
    df_File.loc[[i], "pl_daily"] = df_File["pl_ltd_opcao"][i] - df_File["d1_pl_ltd_opcao"][i]
    df_File.loc[[i], "pl_month"] = df_File["pl_ltd_opcao"][i] - df_File["m1_pl_ltd_opcao"][i]

    # Inicio do calculo do campo: d0_pl_exposure_ibov
    if (df_File["Data_Liquidacao"][i] is None):
        df_File.loc[[i], "d0_pl_exposure_ibov"] = 0
    else:
        if(v_Data_base_d0 < df_File["Trade_Date"][i] or v_Data_base_d0 >= df_File["Vencimento"][i]):
            df_File.loc[[i], "d0_pl_exposure_ibov"] = 0
        else:
            df_File.loc[[i], "d0_pl_exposure_ibov"] = ((df_File["Notional"][i]/df_File["Strike"][i]))*df_File["d1_delta_perc"][i]*(df_File["m_indice_futuro"][i]-df_File["d1_m_indice_futuro"][i])
    # Fim do calculo do campo: d0_pl_exposure_ibov
    
    
    # Inicio do calculo do campo: d0_pL_Theta
    
    df_File.loc[[i], "d0_pL_Theta"] = df_File["d1_theta"][i]
    
    # Fim do calculo do campo: d0_pL_Theta

    # Inicio do calculo do campo: d0_pL_vega_um_perc_IBOV
    if (df_File["Trade_Date"][i] is None or df_File["Vencimento"][i] is None):
        df_File.loc[[i], "d0_pL_vega_um_perc_IBOV"] = 0
    else:
        if (v_Data_base_d0 < df_File["Trade_Date"][i] or v_Data_base_d0 >= df_File["Vencimento"][i]):
            df_File.loc[[i], "d0_pL_vega_um_perc_IBOV"] = 0
        else:
            df_File.loc[[i], "d0_pL_vega_um_perc_IBOV"] = df_File["d1_vega_perc_um_IBOV"][i] *(df_File["m_vol_perc"][i]-df_File["d1_m_vol_perc"][i])*100

    # Fim do calculo do campo: d0_pL_vega_um_perc_IBOV
    
    
    # Inicio do calculo do campo: d0_pL_gamma_um_perc_IBOV
    
    if(df_File["d1_m_indice_futuro"][i] == 0):
        0
    else:
        if (df_File["Trade_Date"][i] is None or df_File["Vencimento"][i] is None):
            df_File.loc[[i], "d0_pL_gamma_um_perc_IBOV"] = 0
        else:
            if (v_Data_base_d0 <  df_File["Trade_Date"][i] or v_Data_base_d0 >= df_File["Vencimento"][i]):
                df_File.loc[[i], "d0_pL_gamma_um_perc_IBOV"] = 0
            else:
                df_File.loc[[i], "d0_pL_gamma_um_perc_IBOV"] = (df_File["m_indice_futuro"][i]/df_File["d1_m_indice_futuro"][i]-1)*100*df_File["d1_gamma_perc_IBOV"][i]/df_File["Strike"][i]*(df_File["m_indice_futuro"][i]-df_File["d1_m_indice_futuro"][i])/2
    
    # Fim do calculo do campo: d0_pL_gamma_um_perc_IBOV    
    
    
    # Inicio do calculo do campo: d0_pL_Intraday
    
    if (df_File["Trade_Date"][i] is None):
        df_File.loc[[i], "d0_pL_Intraday"] = 0
    else:
        if(df_File["Trade_Date"][i] == v_Data_base_d0):
            df_File.loc[[i], "d0_pL_Intraday"] = df_File["pl_ltd_opcao"][i]
        else:
            df_File.loc[[i], "d0_pL_Intraday"] = 0
   
    # Fim do calculo do campo: d0_pL_Intraday    
    

    # Inicio do calculo do campo: d0_pL_Estimado
    
    df_File.loc[[i], "d0_pL_Estimado"] = df_File["d0_pl_exposure_ibov"][i] + df_File["d0_pL_Theta"][i] + df_File["d0_pL_vega_um_perc_IBOV"][i] + df_File["d0_pL_gamma_um_perc_IBOV"][i] + df_File["d0_pL_Intraday"][i]
   
    # Fim do calculo do campo: d0_pL_Estimado

    # Inicio do calculo do campo: d0_residuo
    if (df_File["Trade_Date"][i] is None):
        df_File.loc[[i], "d0_residuo"] = 0
    else:
        if( v_Data_base_d1 < df_File["Trade_Date"][i]):
            df_File.loc[[i], "d0_residuo"] = 0
        else:
            df_File.loc[[i], "d0_residuo"] = df_File["pl_daily"][i] - df_File["d0_pL_Estimado"][i]
    
    # Fim do calculo do campo: d0_residuo    
 
    # Inicio do calculo do campo: d0_residuo_perc

    if(df_File["pl_daily"][i] == 0):
        df_File.loc[[i], "d0_residuo_perc"] = 0
    else:
        if(df_File["d0_residuo"][i] == 0):
            df_File.loc[[i], "d0_residuo_perc"] = 0
        else:
            df_File.loc[[i], "d0_residuo_perc"] = (df_File["d0_residuo"][i] / df_File["pl_daily"][i])
    
    # Fim do calculo do campo: d0_residuo_perc 
    
    # Inicio do calculo do campo: d0_notional

    if(df_File["Data_Liquidacao"][i] is None):
        df_File.loc[[i], "d0_notional"] = 0
    else:
        if(df_File["Liquidacao_Antecipada_x"][i] == "x" and df_File["Data_Liquidacao"][i] <= v_Data_base_d0):
            df_File.loc[[i], "d0_notional"] = 0
        else:
            df_File.loc[[i], "d0_notional"] = df_File["Notional"][i]
    
    # Fim do calculo do campo: d0_notional 

    # Inicio do calculo do campo: d0_delta_perc
    
    if(df_File["Trade_Date"][i] is None or df_File["Vencimento"][i] is None or df_File["Data_Liquidacao"][i] is None):
        df_File.loc[[i], "d0_delta_perc"] = 0
    else:
        if(v_Data_base_d0 < df_File["Trade_Date"][i] or v_Data_base_d0 <= df_File["Vencimento"][i] or ( df_File["Liquidacao_Antecipada_x"][i] == "x" and df_File["Data_Liquidacao"][i] <= v_Data_base_d0)):
            df_File.loc[[i], "d0_delta_perc"] = 0
        else:
            df_File.loc[[i], "d0_delta_perc"] = DERIVATIVOS_DELTA_OPCAO_IBOV(v_Tipo,v_FWD, v_strike, v_Vol ,0, v_T)

    # Fim do calculo do campo: d0_delta_perc 

    # Inicio do calculo do campo: d0_exposure_IBOV
    if (df_File["Data_Liquidacao"][i] is None):
        df_File.loc[[i], "d0_exposure_IBOV"] = 0
    else:
        if(df_File["Liquidacao_Antecipada_x"][i] == "x" and df_File["Data_Liquidacao"][i] <= v_Data_base_d0):
            df_File.loc[[i], "d0_exposure_IBOV"] = 0
        else:
            df_File.loc[[i], "d0_exposure_IBOV"] = df_File["d0_notional"][i] * df_File["d0_delta_perc"][i]

    # Fim do calculo do campo: d0_exposure_IBOV 
    
    # Inicio do calculo do campo: d0_theta
    
    # Condição para que seja carregada a variavel que será inserida na chamada da função DERIVATIVOS_OPCAO_IBOV
    if(df_File["m_data_base_vencimento_du"][i] is None):
        v_m_data_base_vencimento_du is None
    else:
        v_m_data_base_vencimento_du = df_File["m_data_base_vencimento_du"][i] - 1 
    
    if (df_File["Data_Liquidacao"][i] is None):
        df_File.loc[[i], "d0_theta"] = 0
    else:
        if(v_Data_base_d0 < df_File["Trade_Date"][i]):
            df_File.loc[[i], "d0_theta"] = 0
        else:
            if(df_File["Liquidacao_Antecipada_x"][i] == "x" and df_File["Data_Liquidacao"][i] <= v_Data_base_d0):
                df_File.loc[[i], "d0_theta"] = 0
            else:
                df_File.loc[[i], "d0_theta"] = DERIVATIVOS_OPCAO_IBOV(v_Tipo, v_FWD, v_strike, v_Vol, 0, v_T)-df_File["m_preco_opcao"][i] * V_Notional / v_strike

    # Fim do calculo do campo: d0_theta     
    
    # Inicio do calculo do campo: d0_vega_perc_um_IBOV

    if (v_Data_base_d0 < df_File["Trade_Date"][i]):
        df_File.loc[[i], "d0_vega_perc_um_IBOV"] = 0
    else:
        if (df_File["Liquidacao_Antecipada_x"][i] == "x" and df_File["Data_Liquidacao"][i] <= v_Data_base_d0):
            df_File.loc[[i], "d0_vega_perc_um_IBOV"] = 0
        else:
            df_File.loc[[i], "d0_vega_perc_um_IBOV"] = (DERIVATIVOS_OPCAO_IBOV(v_Tipo, v_FWD, v_strike, v_Vol + 0.01, 0, v_T) - DERIVATIVOS_OPCAO_IBOV(v_Tipo, v_FWD, v_strike, v_Vol - 0.01, 0, v_T)) / 2 * V_Notional / v_strike

    # Fim do calculo do campo: d0_vega_perc_um_IBOV

    
    # Inicio do calculo do campo: d0_gamma_perc_IBOV
    
    # Possui a mesma regra do campo d0_vega_perc_um_IBOV
    df_File.loc[[i], "d0_gamma_perc_IBOV"] = df_File["d0_vega_perc_um_IBOV"][i]
    
    # Fim do calculo do campo: d0_gamma_perc_IBOV
    
    # Inicio do calculo do campo: d0_data
    
    df_File.loc[[i], "d0_data"] = df_File["Vencimento"][i]
    
    # Fim do calculo do campo: d0_data    


Gerar o arquivo que será enviado para o Cloud Storage.

In [287]:
create_dataframe_to_csv(df_File, v_target_file_ic, v_upload_file_name)

Faz o upload dos dados para o CS.

In [288]:
upload_file_cs(v_bucket_name, v_source_file_ic + v_upload_file_name, v_target_file_ic + v_upload_file_name )